# 02 - Data Preprocessing

**Purpose**: Clean data, encode, scale, split, and SAVE for next notebook.

**Outputs**:
- `data/processed/features_v1_train.csv`
- `data/processed/features_v1_test.csv`
- `models/scaler_v1.joblib`

In [10]:
import sys; sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

In [11]:
# Paths
DATA_RAW_PATH = '../data/raw/cancer_endometri.csv'
DATA_PROCESSED_PATH = '../data/processed/'
MODELS_PATH = '../models/'

# Target
TARGET = 'recidiva_exitus'

# Columnes a eliminar (leakage + soroll)
LEAKAGE_COLUMNS = [
    'recid_super_1', 'recidiva', 'fecha_de_recidi', 'f_muerte', 'causa_muerte',
    'tto_recidiva', 'Tt_recidiva_qx', 'otro_ttIQ_recid',
    'loc_recidiva_r01', 'loc_recidiva_r02', 'loc_recidiva_r03',
    'loc_recidiva_r04', 'loc_recidiva_r05', 'loc_recidiva_r06',
    'numero_de_recid', 'num_recidiva', 'dx_recidiva', 'libre_enferm',
    'est_pcte', 'estado', 'visita_control', 'Ultima_fecha',
    'diferencia_dias_reci_exit', 'despues_diag',
    'codigo_participante', 'usuario_reg1', 'f_diag', 'FN', 'fecha_qx', 'f_1v',
    'f_tto_NA', 'comentarios', 'otras_especifi', 'otra_histo', 'histo_otros',
    'ap_comentarios', 'inicio_qmt', 'fecha_final_qmt', 'tt_o_f_ini', 'tt_o_f_fin',
    'ini_bqt_rt', 'final_bqt_rt', 'ap_gPelv_loc', 'Tributaria_a_Radioterapia',
    'otros_tt', 'qt', 'bqt', 'moti_no_RT', 'rdt', 'estadificacion_',
    'dias_de_ingreso', 'asa', 'tiempo_qx', 'centro_tratPrima', 'motivonolaparos',
    'trazador_utiliz', 'tabla_de_estadi', 'tabla_de_riesgo',
    'compl_precoc_r01', 'compl_precoc_r02', 'compl_precoc_r03',
    'compl_precoc_r04', 'compl_precoc_r05', 'compl_precoc_r06',
    'compl_precoc_r07', 'compl_precoc_r08', 'compl_precoc_r09',
    'compl_precoc_r10', 'compl_precoc_r11', 'compl_precoc_r12',
    'compl_precoc_r13', 'compl_precoc_r14',
    'comp_intraop_r01', 'comp_intraop_r02', 'comp_intraop_r03',
    'comp_intraop_r04', 'comp_intraop_r05', 'comp_intraop_r06',
    'comp_intraop_r07', 'otras', 'comp_claviendin_mes',
    'reintervencion', 'reintervencion_motivo', 'tiempo_transcur',
    'perdida_hematic', 'perdida_hem_cc', 'n_resec_Intes', 'oment_Avan',
    'ciclos_tto_NAdj', 'transf_GRC',
    'estudio_genetico_r01', 'estudio_genetico_r02', 'estudio_genetico_r03',
    'estudio_genetico_r04', 'estudio_genetico_r05', 'estudio_genetico_r06',
    'ultraestidaije_GC', 'inf_param_vag',
    'p53_molecular', 'p53_ihq', 'mut_pole', 'msh2', 'msh6', 'pms2', 'mlh1',
    'Reseccion_macroscopica_complet', 'Tratamiento_RT', 'Tratamiento_sistemico',
    'presntado_cTG', 
]

# Prefixos a excloure
PREFIXES_TO_EXCLUDE = ['tec_', 'gc_', 'Motivo_de_conversion_']

In [12]:
# Carregar dades
df = pd.read_csv(DATA_RAW_PATH)

# Netejar noms de columnes (eliminar espais)
df.columns = df.columns.str.strip()

print(f"Dades carregades: {df.shape[0]} files x {df.shape[1]} columnes")

# Filtrar columnes (eliminar leakage i soroll)
columns_to_keep = [
    col for col in df.columns 
    if col not in LEAKAGE_COLUMNS 
    and not any(col.startswith(prefix) for prefix in PREFIXES_TO_EXCLUDE)
]

df = df[columns_to_keep].copy()

print(f"Després de filtrar columnes: {df.shape[0]} files x {df.shape[1]} columnes")
print(f"\nColumnes disponibles: {list(df.columns)}")

Dades carregades: 163 files x 189 columnes
Després de filtrar columnes: 163 files x 56 columnes

Columnes disponibles: ['recidiva_exitus', 'edad', 'imc', 'tipo_histologico', 'Grado', 'valor_de_ca125', 'ecotv_infiltsub', 'ecotv_infiltobj', 'metasta_distan', 'grupo_riesgo', 'estadiaje_pre_i', 'hsp_trat_primario', 'tto_NA', 'tto_1_quirugico', 'inten_tto', 'abordajeqx', 'conver_laparo', 'Tec_histerec', 'Anexectomia', 'omentectomia', 'Perforacion_uterina', 'Movilizador_uterino', 'tc_gc', 'histe_avanz', 'histo_defin', 'grado_histologi', 'tamano_tumoral', 'infilt_estr_cervix', 'infiltracion_mi', 'tx_anexial', 'tx_sincronico', 'afectacion_linf', 'afectacion_omen', 'AP_centinela_pelvico', 'n_total_GC', 'n_GC_Afect', 'AP_ganPelv', 'n_total_ganCent', 'n_gangP_afec', 'AP_glanPaor', 'Local_Gan_Paor', 'n_total_ganPaor_infra', 'n_ganPaor_InfrM_afec', 'n_total_ganPaor_supr', 'n_ganPaor_Sup_afec', 'ap_gPaor_total', 'ap_gPor_afect_tot', 'recep_est_porcent', 'rece_de_Ppor', 'beta_cateninap', 'FIGO2023', 

In [13]:
# Eliminar files on recidiva_exitus és NA o 2 (Desconegut)
df = df[df[TARGET].notna()]
df = df[df[TARGET] != 2]

# Convertir target a int
df[TARGET] = df[TARGET].astype(int)

print(f"Després de filtrar target: {df.shape[0]} files x {df.shape[1]} columnes")
print(f"\nDistribució del target:\n{df[TARGET].value_counts()}")

Després de filtrar target: 154 files x 56 columnes

Distribució del target:
recidiva_exitus
0    120
1     34
Name: count, dtype: int64


In [14]:
# Veure quines columnes tenen NAs i quin percentatge
na_percent = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
cols_with_na = na_percent[na_percent > 0]
print(f"Columnes amb NAs ({len(cols_with_na)}):")
for col, pct in cols_with_na.items():
    print(f"  {col}: {pct:.1f}%")

Columnes amb NAs (50):
  histe_avanz: 90.9%
  valor_de_ca125: 86.4%
  Tratamiento_sistemico_realizad: 81.2%
  AP_glanPaor: 79.2%
  rt_dosis: 79.2%
  Local_Gan_Paor: 79.2%
  bt_realPac: 76.0%
  AP_ganPelv: 64.9%
  n_doisis_rt: 51.3%
  rece_de_Ppor: 44.8%
  recep_est_porcent: 43.5%
  n_ganPaor_Sup_afec: 39.0%
  n_ganPaor_InfrM_afec: 38.3%
  n_total_ganPaor_supr: 35.1%
  n_total_ganPaor_infra: 34.4%
  ap_gPor_afect_tot: 29.9%
  ap_gPaor_total: 26.6%
  n_gangP_afec: 25.3%
  afectacion_omen: 22.7%
  n_total_ganCent: 20.8%
  conver_laparo: 19.5%
  tamano_tumoral: 18.2%
  n_GC_Afect: 18.2%
  omentectomia: 17.5%
  AP_centinela_pelvico: 17.5%
  Movilizador_uterino: 17.5%
  Perforacion_uterina: 16.2%
  n_total_GC: 16.2%
  Anexectomia: 15.6%
  Tec_histerec: 15.6%
  tc_gc: 14.3%
  FIGO2023: 11.7%
  grupo_de_riesgo_definitivo: 10.4%
  afectacion_linf: 10.4%
  tx_anexial: 9.7%
  grado_histologi: 9.7%
  abordajeqx: 8.4%
  infiltracion_mi: 8.4%
  beta_cateninap: 7.1%
  infilt_estr_cervix: 6.5%
  tx_si

In [15]:
# histe_avanz (90.9% NA)
# Lògica: 0 → 0, NaN → 1
def impute_histe_avanz(val):
    if val == 0:
        return 0
    elif pd.isna(val) or val != 2:
        return 1
    else:
        return val  # Manté el 2 si existeix

df['histe_avanz'] = df['histe_avanz'].apply(impute_histe_avanz)

print("Resultat imputació histe_avanz:")
print(df['histe_avanz'].value_counts(dropna=False))
print(f"NAs restants: {df['histe_avanz'].isna().sum()}")

# valor_de_ca125 (86.4% NA)
# Categorització: <35 = Normal, >=35 = Elevado, NA = Desconocido
# Llindar clínic: 35 U/mL
def categorize_ca125(val):
    if pd.isna(val):
        return "Desconegut"
    elif val < 35:
        return "Normal"
    else:
        return "Elevat"

df['valor_de_ca125'] = df['valor_de_ca125'].apply(categorize_ca125)

print("\nResultat categorització valor_de_ca125:")
print(df['valor_de_ca125'].value_counts(dropna=False))

Resultat imputació histe_avanz:
histe_avanz
1.0    140
0.0     13
2.0      1
Name: count, dtype: int64
NAs restants: 0

Resultat categorització valor_de_ca125:
valor_de_ca125
Desconegut    133
Normal         11
Elevat         10
Name: count, dtype: int64


In [16]:
# grado_histologi (9.7% NA) → MODA

# JUSTIFICACIÓ: Necessitem el grau histològic per imputar les variables de
# receptors hormonals (recep_est_porcent, rece_de_Ppor) i tamano_tumoral,
# ja que la mediana estratificada depèn d'aquesta variable.

grado_mode = df['grado_histologi'].mode()[0]
df['grado_histologi'] = df['grado_histologi'].fillna(grado_mode)
print(f"  grado_histologi: imputat amb moda = {grado_mode}")
print(f"    Distribució: {df['grado_histologi'].value_counts().to_dict()}")


# AP_glanPaor (79.2% NA) → -1 (No valorat)

# JUSTIFICACIÓ: La linfadenectomia paraórtica NO es fa a totes les pacients,
# només a les d'alt risc o estadis avançats. L'alt % de NA indica que
# simplement no es va realitzar la prova (no que sigui negativa)

df['AP_glanPaor'] = df['AP_glanPaor'].fillna(-1)
print(f"  AP_glanPaor: imputat amb -1 (No valorat)")
print(f"    Distribució: {df['AP_glanPaor'].value_counts().to_dict()}")

# AP_ganPelv (64.9% NA) → -1 (Desconegut)

# JUSTIFICACIÓ: Mateixa justificació abans
df['AP_ganPelv'] = df['AP_ganPelv'].fillna(-1)
print(f"  AP_ganPelv: imputat amb -1 (Desconegut)")
print(f"    Distribució: {df['AP_ganPelv'].value_counts().to_dict()}")

# AP_centinela_pelvico (17.5% NA) → -1 (No valorat)

# JUSTIFICACIÓ: Mateixa justificació abans

df['AP_centinela_pelvico'] = df['AP_centinela_pelvico'].fillna(-1)
print(f"  AP_centinela_pelvico: imputat amb -1 (No valorat)")
print(f"    Distribució: {df['AP_centinela_pelvico'].value_counts().to_dict()}")


# rt_dosis (79.2% NA) → -1 (Desconegut)

# JUSTIFICACIÓ: La radioteràpia només s'indica en casos específics (risc
# intermedi-alt). L'alt % de NA pot indicar que no es va fer O que no es
# va registrar. Usem -1 per ser conservadors.

df['rt_dosis'] = df['rt_dosis'].fillna(-1)
print(f"  rt_dosis: imputat amb -1 (Desconegut)")
print(f"    Distribució: {df['rt_dosis'].value_counts().to_dict()}")


# =============================================================================
# FASE 2: VARIABLES DEPENDENTS DE GANGLIS PARAÓRTICS (depenen de AP_glanPaor)
# =============================================================================
# Aquestes variables només tenen sentit si es va fer linfadenectomia paraórtica.
# Si AP_glanPaor = -1, les variables de comptatge han de ser 0.
# Si AP_glanPaor = 0 (negatiu), les d'afectació han de ser 0.
# =============================================================================

# Local_Gan_Paor (79.2% NA) → -1 (No aplica)

# JUSTIFICACIÓ: Localització dels ganglis paraórtics afectats. Només té sentit
# si AP_glanPaor > 0 (hi ha afectació). La lògica seria:
# - Si AP_glanPaor ∈ {-1 (no valorat), 0 (negatiu)}: No aplica → -1
# - Si AP_glanPaor > 0 però Local_Gan_Paor és NA: Mantenim -1 (Desconegut)
# -----------------------------------------------------------------------------
df['Local_Gan_Paor'] = df['Local_Gan_Paor'].fillna(-1)
print(f"  Local_Gan_Paor: imputat amb -1 (No aplica)")

# -----------------------------------------------------------------------------
# Variables de comptatge de ganglis paraórtics
# -----------------------------------------------------------------------------
# LÒGICA COMUNA:
# - Si AP_glanPaor = -1 (no es va fer linfadenectomia): Total examinats = 0
# - Si AP_glanPaor ∈ {-1, 0}: Afectats = 0 (no n'hi pot haver si no es van
#   valorar o eren negatius)
# - Si AP_glanPaor > 0 però la variable és NA: -1 (Desconegut)
# -----------------------------------------------------------------------------

# n_total_ganPaor_supr (35.1% NA) - Total ganglis supramesentèrics examinats
def impute_total_ganPaor_supr(row):
    if pd.isna(row['n_total_ganPaor_supr']):
        if row['AP_glanPaor'] == -1:
            return 0  # No es va fer linfadenectomia
        else:
            return -1  # Es va fer però no sabem quants
    return row['n_total_ganPaor_supr']

df['n_total_ganPaor_supr'] = df.apply(impute_total_ganPaor_supr, axis=1)
print(f"  n_total_ganPaor_supr: imputat condicionalment (0 si no valorat, -1 si desconegut)")

# n_total_ganPaor_infra (34.4% NA) - Total ganglis inframesentèrics examinats
def impute_total_ganPaor_infra(row):
    if pd.isna(row['n_total_ganPaor_infra']):
        if row['AP_glanPaor'] == -1:
            return 0
        else:
            return -1
    return row['n_total_ganPaor_infra']

df['n_total_ganPaor_infra'] = df.apply(impute_total_ganPaor_infra, axis=1)
print(f"  n_total_ganPaor_infra: imputat condicionalment")

# ap_gPaor_total (26.6% NA) - Total ganglis paraórtics examinats
def impute_gPaor_total(row):
    if pd.isna(row['ap_gPaor_total']):
        if row['AP_glanPaor'] == -1:
            return 0
        else:
            return -1
    return row['ap_gPaor_total']

df['ap_gPaor_total'] = df.apply(impute_gPaor_total, axis=1)
print(f"  ap_gPaor_total: imputat condicionalment")

# n_ganPaor_Sup_afec (39.0% NA) - Ganglis supramesentèrics AFECTATS
def impute_ganPaor_Sup_afec(row):
    if pd.isna(row['n_ganPaor_Sup_afec']):
        if row['AP_glanPaor'] in [-1, 0]:
            return 0  # No valorat o negatiu = 0 afectats
        else:
            return -1
    return row['n_ganPaor_Sup_afec']

df['n_ganPaor_Sup_afec'] = df.apply(impute_ganPaor_Sup_afec, axis=1)
print(f"  n_ganPaor_Sup_afec: imputat condicionalment")

# n_ganPaor_InfrM_afec (38.3% NA) - Ganglis inframesentèrics AFECTATS
def impute_ganPaor_InfrM_afec(row):
    if pd.isna(row['n_ganPaor_InfrM_afec']):
        if row['AP_glanPaor'] in [-1, 0]:
            return 0
        else:
            return -1
    return row['n_ganPaor_InfrM_afec']

df['n_ganPaor_InfrM_afec'] = df.apply(impute_ganPaor_InfrM_afec, axis=1)
print(f"  n_ganPaor_InfrM_afec: imputat condicionalment")

# ap_gPor_afect_tot (29.9% NA) - Total ganglis paraórtics AFECTATS
def impute_gPor_afect_tot(row):
    if pd.isna(row['ap_gPor_afect_tot']):
        if row['AP_glanPaor'] in [-1, 0]:
            return 0
        else:
            return -1
    return row['ap_gPor_afect_tot']

df['ap_gPor_afect_tot'] = df.apply(impute_gPor_afect_tot, axis=1)
print(f"  ap_gPor_afect_tot: imputat condicionalment")

print()

# =============================================================================
# FASE 3: VARIABLES DEPENDENTS DE GANGLIS PÈLVICS (depenen de AP_ganPelv)
# =============================================================================

print("FASE 3: Imputant variables dependents de ganglis pèlvics...")
print("-"*40)

# -----------------------------------------------------------------------------
# 3.1 n_gangP_afec (25.3% NA) - Nº ganglis pèlvics afectats
# -----------------------------------------------------------------------------
# LÒGICA: Si AP_ganPelv = -1 o 0, no hi pot haver ganglis afectats
# -----------------------------------------------------------------------------
def impute_gangP_afec(row):
    if pd.isna(row['n_gangP_afec']):
        if row['AP_ganPelv'] in [-1, 0]:
            return 0
        else:
            return -1
    return row['n_gangP_afec']

df['n_gangP_afec'] = df.apply(impute_gangP_afec, axis=1)
print(f"  n_gangP_afec: imputat condicionalment")

print()

# =============================================================================
# FASE 4: VARIABLES DEPENDENTS DE GANGLIO CENTINELA (depenen de AP_centinela_pelvico)
# =============================================================================

print("FASE 4: Imputant variables dependents de ganglio centinela...")
print("-"*40)

# n_total_ganCent (20.8% NA) - Total ganglis centinela examinats
def impute_total_ganCent(row):
    if pd.isna(row['n_total_ganCent']):
        if row['AP_centinela_pelvico'] == -1:
            return 0  # No es va fer GC
        else:
            return -1
    return row['n_total_ganCent']

df['n_total_ganCent'] = df.apply(impute_total_ganCent, axis=1)
print(f"  n_total_ganCent: imputat condicionalment")

# n_GC_Afect (18.2% NA) - Nº ganglis centinela afectats
def impute_GC_Afect(row):
    if pd.isna(row['n_GC_Afect']):
        if row['AP_centinela_pelvico'] in [-1, 0]:
            return 0  # No valorat o negatiu = 0 afectats
        else:
            return -1
    return row['n_GC_Afect']

df['n_GC_Afect'] = df.apply(impute_GC_Afect, axis=1)
print(f"  n_GC_Afect: imputat condicionalment")

# n_total_GC (16.2% NA) - Total ganglis centinela
def impute_total_GC(row):
    if pd.isna(row['n_total_GC']):
        if row['AP_centinela_pelvico'] == -1:
            return 0
        else:
            return -1
    return row['n_total_GC']

df['n_total_GC'] = df.apply(impute_total_GC, axis=1)
print(f"  n_total_GC: imputat condicionalment")

print()

# =============================================================================
# FASE 5: VARIABLES DEPENDENTS DE RADIOTERÀPIA (depenen de rt_dosis)
# =============================================================================

print("FASE 5: Imputant variables dependents de radioteràpia...")
print("-"*40)

# -----------------------------------------------------------------------------
# 5.1 n_doisis_rt (51.3% NA) - Nombre de dosis de RT
# -----------------------------------------------------------------------------
# LÒGICA: Si rt_dosis = -1 (desconegut) o 0 (no realitzada), les dosis = 0
# Si rt_dosis > 0 (es va fer RT) però n_doisis_rt és NA, és desconegut
# -----------------------------------------------------------------------------
def impute_n_dosis_rt(row):
    if pd.isna(row['n_doisis_rt']):
        if row['rt_dosis'] in [-1, 0]:
            return 0  # No es va fer RT o és desconegut
        else:
            return -1  # Es va fer RT però no sabem quantes dosis
    return row['n_doisis_rt']

df['n_doisis_rt'] = df.apply(impute_n_dosis_rt, axis=1)
print(f"  n_doisis_rt: imputat condicionalment")

print()

# =============================================================================
# FASE 6: VARIABLES NUMÈRIQUES AMB MEDIANA ESTRATIFICADA
# =============================================================================
# Per a variables numèriques contínues, la mediana és més robusta que la mitjana.
# L'estratificació per grado_histologi és clínicament rellevant perquè:
# - Grau 1 (baix): Tumors més diferenciats, més receptors hormonals, més petits
# - Grau 2 (alt): Tumors menys diferenciats, menys receptors, més agressius
# =============================================================================

print("FASE 6: Imputant variables numèriques amb mediana estratificada...")
print("-"*40)

# -----------------------------------------------------------------------------
# 6.1 recep_est_porcent (43.5% NA) - Receptors d'Estrogen (%)
# -----------------------------------------------------------------------------
# VARIABLE ESTRELLA per al perfil NSMP. Alta expressió = millor pronòstic.
# Estratificació per grau: G1 sol tenir més receptors que G2.
# -----------------------------------------------------------------------------
median_er_by_grade = df.groupby('grado_histologi')['recep_est_porcent'].median()
print(f"  recep_est_porcent: medianes per grau = {median_er_by_grade.to_dict()}")

def impute_recep_est(row):
    if pd.isna(row['recep_est_porcent']):
        grade = row['grado_histologi']
        if grade in median_er_by_grade.index:
            return median_er_by_grade[grade]
        else:
            return df['recep_est_porcent'].median()  # Fallback
    return row['recep_est_porcent']

df['recep_est_porcent'] = df.apply(impute_recep_est, axis=1)
# Fallback per si encara hi ha NAs
df['recep_est_porcent'] = df['recep_est_porcent'].fillna(df['recep_est_porcent'].median())
print(f"    Imputat amb mediana estratificada per grau histològic")

# -----------------------------------------------------------------------------
# 6.2 rece_de_Ppor (44.8% NA) - Receptors de Progesterona (%)
# -----------------------------------------------------------------------------
# Mateixa lògica que recep_est_porcent. Normalment ER i PR estan correlacionats.
# -----------------------------------------------------------------------------
median_pr_by_grade = df.groupby('grado_histologi')['rece_de_Ppor'].median()
print(f"  rece_de_Ppor: medianes per grau = {median_pr_by_grade.to_dict()}")

def impute_rece_de_Ppor(row):
    if pd.isna(row['rece_de_Ppor']):
        grade = row['grado_histologi']
        if grade in median_pr_by_grade.index:
            return median_pr_by_grade[grade]
        else:
            return df['rece_de_Ppor'].median()
    return row['rece_de_Ppor']

df['rece_de_Ppor'] = df.apply(impute_rece_de_Ppor, axis=1)
df['rece_de_Ppor'] = df['rece_de_Ppor'].fillna(df['rece_de_Ppor'].median())
print(f"    Imputat amb mediana estratificada per grau histològic")

# -----------------------------------------------------------------------------
# 6.3 tamano_tumoral (18.2% NA) - Mida del tumor (cm)
# -----------------------------------------------------------------------------
# Els tumors d'alt grau solen ser més grans. Estratifiquem per grau.
# -----------------------------------------------------------------------------
median_size_by_grade = df.groupby('grado_histologi')['tamano_tumoral'].median()
print(f"  tamano_tumoral: medianes per grau = {median_size_by_grade.to_dict()}")

def impute_tamano_tumoral(row):
    if pd.isna(row['tamano_tumoral']):
        grade = row['grado_histologi']
        if grade in median_size_by_grade.index:
            return median_size_by_grade[grade]
        else:
            return df['tamano_tumoral'].median()
    return row['tamano_tumoral']

df['tamano_tumoral'] = df.apply(impute_tamano_tumoral, axis=1)
df['tamano_tumoral'] = df['tamano_tumoral'].fillna(df['tamano_tumoral'].median())
print(f"    Imputat amb mediana estratificada per grau histològic")

# -----------------------------------------------------------------------------
# 6.4 imc (3.9% NA) - Índex de Massa Corporal
# -----------------------------------------------------------------------------
# No hi ha relació directa amb el grau. Usem mediana global.
# L'obesitat és factor de risc per a CE, però no afecta el pronòstic directament.
# -----------------------------------------------------------------------------
imc_median = df['imc'].median()
df['imc'] = df['imc'].fillna(imc_median)
print(f"  imc: imputat amb mediana global = {imc_median:.2f}")

print()

# =============================================================================
# FASE 7: VARIABLES BINÀRIES QUIRÚRGIQUES → 0 (No)
# =============================================================================
# Aquestes variables representen complicacions, afectacions o procediments
# que normalment es REGISTREN quan passen. Si no hi ha dada, assumim que NO
# va passar (el protocol mèdic sol registrar les excepcions).
# =============================================================================

print("FASE 7: Imputant variables binàries quirúrgiques amb 0 (No)...")
print("-"*40)

# Llista de variables binàries on NA → 0 (No)
binary_zero_vars = {
    'afectacion_omen': 'Afectació omental: si fos positiva, s\'hauria registrat',
    'conver_laparo': 'Conversió a laparotomia: és excepcional i sempre es registra',
    'omentectomia': 'Omentectomia: només en estadis avançats, s\'hauria registrat',
    'Perforacion_uterina': 'Perforació uterina: complicació que sempre es registra',
    'afectacion_linf': 'LVSI: si fos positiva, s\'hauria registrat al informe AP',
    'tx_anexial': 'Tumor anexial sincrònic: és rar i s\'hauria registrat',
    'infilt_estr_cervix': 'Infiltració cervical: important per estadiatge, s\'hauria registrat',
    'tx_sincronico': 'Tumor sincrònic: és excepcional i sempre es registra',
    'metasta_distan': 'Metàstasi a distància: crític, sempre es registra si existeix',
}

for var, justificacio in binary_zero_vars.items():
    na_count = df[var].isna().sum()
    df[var] = df[var].fillna(0)
    print(f"  {var}: {na_count} NAs → 0")
    print(f"    Justificació: {justificacio}")

print()

# =============================================================================
# FASE 8: VARIABLES QUIRÚRGIQUES AMB MODA
# =============================================================================
# Per a variables categòriques on el valor més freqüent és una bona aproximació
# del valor més probable per als missings.
# =============================================================================

print("FASE 8: Imputant variables quirúrgiques amb moda...")
print("-"*40)

# Annexectomia i Histerectomia
# Justificació: Són procediments estàndard en CE. La majoria de pacients els reben.
for var in ['Anexectomia', 'Tec_histerec']:
    mode_val = df[var].mode()[0]
    na_count = df[var].isna().sum()
    df[var] = df[var].fillna(mode_val)
    print(f"  {var}: {na_count} NAs → moda = {mode_val}")

# Abordatge quirúrgic
# Justificació: Assumim que el més freqüent és laparoscòpia (estàndard actual)
abordajeqx_mode = df['abordajeqx'].mode()[0]
df['abordajeqx'] = df['abordajeqx'].fillna(abordajeqx_mode)
print(f"  abordajeqx: NAs → moda = {abordajeqx_mode}")

print()

# =============================================================================
# FASE 9: VARIABLES CLÍNIQUES/PATOLÒGIQUES AMB MODA
# =============================================================================

print("FASE 9: Imputant variables clíniques/patològiques amb moda...")
print("-"*40)

mode_clinical_vars = [
    'FIGO2023',                    # Estadiatge FIGO 2023
    'grupo_de_riesgo_definitivo',  # Grup de risc definitiu
    'infiltracion_mi',             # Infiltració miometrial
    'histo_defin',                 # Histologia definitiva
    'ecotv_infiltobj',             # Eco TV infiltració objectiva
    'ecotv_infiltsub',             # Eco TV infiltració subjectiva
    'estadiaje_pre_i',             # Estadiatge prequirúrgic
    'grupo_riesgo',                # Grup de risc preoperatori
]

for var in mode_clinical_vars:
    if df[var].isna().sum() > 0:
        mode_val = df[var].mode()[0]
        na_count = df[var].isna().sum()
        df[var] = df[var].fillna(mode_val)
        print(f"  {var}: {na_count} NAs → moda = {mode_val}")

print()

# =============================================================================
# FASE 10: VARIABLES AMB CATEGORIA "DESCONEGUT" (-1)
# =============================================================================
# Per a variables on no podem assumir res i és més honest dir "no ho sabem".
# =============================================================================

print("FASE 10: Imputant variables amb categoria 'Desconegut' (-1)...")
print("-"*40)

unknown_vars = {
    'Movilizador_uterino': 'Ús de movilitzador: varia segons centre/cirurgià',
    'tc_gc': 'Tècnica ganglio centinela: no podem assumir quin tipus',
    'bt_realPac': 'Braquiteràpia realitzada: no podem assumir si es va fer',
}

for var, justificacio in unknown_vars.items():
    na_count = df[var].isna().sum()
    df[var] = df[var].fillna(-1)
    print(f"  {var}: {na_count} NAs → -1 (Desconegut)")
    print(f"    Justificació: {justificacio}")

print()

# =============================================================================
# FASE 11: VARIABLES DE TRACTAMENT SISTÈMIC I ADJUVANT
# =============================================================================

print("FASE 11: Imputant variables de tractament...")
print("-"*40)

# -----------------------------------------------------------------------------
# Tratamiento_sistemico_realizad (81.2% NA) → 0 (No realizada)
# -----------------------------------------------------------------------------
# JUSTIFICACIÓ: La quimioteràpia adjuvant només s'indica en alt risc.
# L'alt % de NA suggereix que simplement no es va fer (no es registra quan no es fa).
# -----------------------------------------------------------------------------
df['Tratamiento_sistemico_realizad'] = df['Tratamiento_sistemico_realizad'].fillna(0)
print(f"  Tratamiento_sistemico_realizad: NAs → 0 (No realizada)")
print(f"    Justificació: QT adjuvant només en alt risc, NA indica que no es va fer")

# -----------------------------------------------------------------------------
# inten_tto (5.2% NA) → 1 (Curativo)
# -----------------------------------------------------------------------------
# JUSTIFICACIÓ: La gran majoria de CE es tracten amb intenció curativa.
# El tractament pal·liatiu és excepcional i es registra explícitament.
# -----------------------------------------------------------------------------
df['inten_tto'] = df['inten_tto'].fillna(1)
print(f"  inten_tto: NAs → 1 (Curativo)")

# -----------------------------------------------------------------------------
# tto_1_quirugico (2.6% NA) → 1 (Sí)
# -----------------------------------------------------------------------------
# JUSTIFICACIÓ: El tractament primari del CE és la cirurgia.
# Les pacients que no es van operar tindrien una anotació específica.
# -----------------------------------------------------------------------------
df['tto_1_quirugico'] = df['tto_1_quirugico'].fillna(1)
print(f"  tto_1_quirugico: NAs → 1 (Sí)")

print()

# =============================================================================
# FASE 12: MARCADOR MOLECULAR
# =============================================================================

print("FASE 12: Imputant marcadors moleculars...")
print("-"*40)

# -----------------------------------------------------------------------------
# beta_cateninap (7.1% NA) → 2 (No realizado)
# -----------------------------------------------------------------------------
# JUSTIFICACIÓ: La determinació de beta-catenina no és estàndard.
# Si no hi ha dada, el més probable és que no es va fer la prova.
# -----------------------------------------------------------------------------
df['beta_cateninap'] = df['beta_cateninap'].fillna(2)
print(f"  beta_cateninap: NAs → 2 (No realizado)")

print()

# =============================================================================
# VERIFICACIÓ FINAL
# =============================================================================

print("="*60)
print("VERIFICACIÓ FINAL")
print("="*60)

# Comptem NAs per columna
na_final = df.isna().sum()
na_remaining = na_final[na_final > 0]

print(f"NAs totals després d'imputar: {na_final.sum()}")
print()

if len(na_remaining) == 0:
    print("✅ TOTES les columnes han estat imputades correctament!")
else:
    print("⚠️ Columnes amb NAs restants:")
    for col, count in na_remaining.items():
        print(f"  {col}: {count} ({count/len(df)*100:.1f}%)")

print()
print("="*60)
print("RESUM D'ESTRATÈGIES UTILITZADES")
print("="*60)
print("""
1. VARIABLES "PARE" (imputades primer):
   - grado_histologi, AP_glanPaor, AP_ganPelv, AP_centinela_pelvico, rt_dosis
   
2. IMPUTACIÓ CONDICIONAL (depenen d'altres variables):
   - Ganglis paraórtics: depenen de AP_glanPaor
   - Ganglis pèlvics: depenen de AP_ganPelv
   - Ganglio centinela: depenen de AP_centinela_pelvico
   - Dosis RT: depèn de rt_dosis
   
3. MEDIANA ESTRATIFICADA (variables numèriques):
   - recep_est_porcent, rece_de_Ppor, tamano_tumoral: per grado_histologi
   - imc: mediana global
   
4. VALOR 0 (assumim "No" si no registrat):
   - Complicacions i afectacions que s'haurien registrat si fossin positives
   
5. MODA (valor més freqüent):
   - Variables categòriques sense dependències clares
   
6. CATEGORIA -1 "DESCONEGUT":
   - Variables on no podem assumir res amb seguretat
""")

  grado_histologi: imputat amb moda = 1.0
    Distribució: {1.0: 126, 2.0: 28}
  AP_glanPaor: imputat amb -1 (No valorat)
    Distribució: {-1.0: 122, 0.0: 24, 3.0: 6, 1.0: 1, 2.0: 1}
  AP_ganPelv: imputat amb -1 (Desconegut)
    Distribució: {-1.0: 100, 0.0: 43, 3.0: 10, 1.0: 1}
  AP_centinela_pelvico: imputat amb -1 (No valorat)
    Distribució: {4.0: 86, 0.0: 37, -1.0: 27, 1.0: 2, 2.0: 1, 3.0: 1}
  rt_dosis: imputat amb -1 (Desconegut)
    Distribució: {-1.0: 122, 2.0: 25, 0.0: 6, 1.0: 1}
  Local_Gan_Paor: imputat amb -1 (No aplica)
  n_total_ganPaor_supr: imputat condicionalment (0 si no valorat, -1 si desconegut)
  n_total_ganPaor_infra: imputat condicionalment
  ap_gPaor_total: imputat condicionalment
  n_ganPaor_Sup_afec: imputat condicionalment
  n_ganPaor_InfrM_afec: imputat condicionalment
  ap_gPor_afect_tot: imputat condicionalment

FASE 3: Imputant variables dependents de ganglis pèlvics...
----------------------------------------
  n_gangP_afec: imputat condicionalment

F

In [17]:
# =============================================================================
# FASE 13: ONE-HOT ENCODING DE VARIABLES CATEGÒRIQUES
# =============================================================================
# Convertim les variables categòriques amb strings a one-hot encoding

print("FASE 13: Aplicant One-Hot Encoding...")
print("-"*40)

# One-Hot Encoding per valor_de_ca125 (Elevat, Desconegut, Normal)
ca125_dummies = pd.get_dummies(df['valor_de_ca125'], prefix='ca125')
print(f"  valor_de_ca125: creades {ca125_dummies.shape[1]} columnes")
print(f"    {list(ca125_dummies.columns)}")

# Afegim les noves columnes al DataFrame
df = pd.concat([df, ca125_dummies], axis=1)

# Eliminem la columna original
df = df.drop(columns=['valor_de_ca125'])

print(f"\n✅ One-Hot Encoding completat!")
print(f"   Noves columnes: ca125_Desconegut, ca125_Elevat, ca125_Normal")
print(f"   Dimensió final: {df.shape[0]} files x {df.shape[1]} columnes")

FASE 13: Aplicant One-Hot Encoding...
----------------------------------------
  valor_de_ca125: creades 3 columnes
    ['ca125_Desconegut', 'ca125_Elevat', 'ca125_Normal']

✅ One-Hot Encoding completat!
   Noves columnes: ca125_Desconegut, ca125_Elevat, ca125_Normal
   Dimensió final: 154 files x 58 columnes


In [18]:
# =============================================================================
# GUARDAR DADES PREPROCESSADES AMB FEATURES SELECCIONADES
# =============================================================================

import os

DATA_PROCESSED_PATH = '../data/processed/'
os.makedirs(DATA_PROCESSED_PATH, exist_ok=True)

# Features seleccionades (les 14 més importants segons Random Forest)
SELECTED_FEATURES = [
    'grupo_de_riesgo_definitivo',
    'afectacion_linf',
    'estadiaje_pre_i',
    'Tratamiento_sistemico_realizad',
    'grado_histologi',
    'infiltracion_mi',
    'FIGO2023',
    'histo_defin',
    'recep_est_porcent',
    'imc',
    'metasta_distan',
    'rece_de_Ppor',
    'tto_1_quirugico',
    'edad'
]

TARGET = 'recidiva_exitus'

# Seleccionar només les columnes necessàries
df_final = df[[TARGET] + SELECTED_FEATURES].copy()

# Guardar el DataFrame amb features seleccionades
df_final.to_csv(f'{DATA_PROCESSED_PATH}preprocessed_v1.csv', index=False)

print(f"✅ Guardat: {DATA_PROCESSED_PATH}preprocessed_v1.csv")
print(f"   {df_final.shape[0]} files x {df_final.shape[1]} columnes")
print(f"   NAs totals: {df_final.isna().sum().sum()}")
print(f"\n📊 Columnes guardades:")
for col in df_final.columns:
    print(f"   - {col}")

✅ Guardat: ../data/processed/preprocessed_v1.csv
   154 files x 15 columnes
   NAs totals: 0

📊 Columnes guardades:
   - recidiva_exitus
   - grupo_de_riesgo_definitivo
   - afectacion_linf
   - estadiaje_pre_i
   - Tratamiento_sistemico_realizad
   - grado_histologi
   - infiltracion_mi
   - FIGO2023
   - histo_defin
   - recep_est_porcent
   - imc
   - metasta_distan
   - rece_de_Ppor
   - tto_1_quirugico
   - edad
